In [ ]:
%load_ext autoreload
%autoreload 2

import env

import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta

# --- 1. Загрузка конфигурации ---
service = env.get_gservice()

if service:
    df_sheet = env.read_df_from_spreadsheet(service, env.SHEET_ID, env.SHEET_NAME)
    print("Данные из Google Sheets загружены")
else:
    raise ConnectionError("Не удалось подключиться к Google API")

RS_TABLE_P7 = 'incent_opex_check_p7'
RS_SCHEMA_P7 = 'ma_data'
ALERT_NAME = "02-incent.p7"

try:
    config_row = df_sheet[df_sheet['name'] == ALERT_NAME].iloc[0]
except IndexError:
    raise ValueError(f"Алерт '{ALERT_NAME}' не найден в Google Sheet")

if config_row['active_flag'] != 'Enabled':
    print(f"Алерт '{ALERT_NAME}' отключен. Пропуск.")
else:
    print(f"Запуск алерта '{ALERT_NAME}'...")

# --- 2. Парсинг параметров ---
ALERT_ACTIVE_FLAG = config_row['active_flag']
THRESHOLD_WARNING = int(config_row['threshold_fixed']) 
THRESHOLD_CRITICAL = int(config_row['threshold_fixed_crit'])
ALERT_CATEGORY = config_row['metric_crit_category']

# Хелпер для SQL списков
def to_sql_list(items):
    if not isinstance(items, list):
        items = [items] 
    if not items:
        return "()"
    
    formatted = []
    for x in items:
        if isinstance(x, str):
            formatted.append(f"'{x}'") 
        else:
            formatted.append(str(x))   
            
    return f"({', '.join(formatted)})"

try:
    # Загружаем JSON настроек
    # Формат: {"partners": {"1000023": "AdJoe", "69680625": "Mistplay", ...}}
    params = json.loads(config_row['config_json'])
    
    PARTNER_NAMES = params['partners']
    CONFIG_PARTNERS = to_sql_list(list(PARTNER_NAMES.keys()))
    
except json.JSONDecodeError as e:
    raise ValueError(f"Ошибка JSON в ячейке config_json: {e}")
except KeyError as e:
    raise ValueError(f"В JSON отсутствует обязательный ключ: {e}")

print(f"Настройки: THRESHOLD_WARNING={THRESHOLD_WARNING} (1 нед.), THRESHOLD_CRITICAL={THRESHOLD_CRITICAL} (4 нед.)")
print(f"Partners: {CONFIG_PARTNERS}")


# --- 3. Функция расчёта периода дат ---

def get_check_period():
    """
    Возвращает (period_start, period_end, extended_start) - периоды для проверки
    
    - period_start, period_end: текущая неделя Пт-Чт (7 дней) для WARNING
    - extended_start: начало 4-недельного периода для CRITICAL
      (текущая неделя + 3 предыдущие = 28 дней)
    
    Логика:
    - Находим последний четверг, для которого прошло СТРОГО БОЛЬШЕ 7 дней
      (т.е. >= 8 дней между четвергом и сегодня)
    - Это гарантирует, что payers_7 полностью "созрели"
    """
    today = datetime.now().date()
    
    # Находим последний четверг
    # weekday: Mon=0, Tue=1, Wed=2, Thu=3, Fri=4, Sat=5, Sun=6
    days_since_thursday = (today.weekday() - 3) % 7
    if days_since_thursday == 0:
        days_since_thursday = 7
    last_thursday = today - timedelta(days=days_since_thursday)
    
    days_passed = (today - last_thursday).days
    if days_passed <= 8:
        last_thursday -= timedelta(days=7)
    
    # Текущая неделя: пятница (start) - четверг (end)
    period_end = last_thursday
    period_start = period_end - timedelta(days=6)
    
    # Расширенный период для CRITICAL: 4 недели (текущая + 3 предыдущие)
    extended_start = period_start - timedelta(days=21)
    
    return period_start, period_end, extended_start


# --- 4. Функция определения типа алерта ---

def get_alert_type(payers_1w, payers_4w):
    """
    Определяет тип алерта на основе количества плательщиков.
    
    - CRITICAL: payers_4w < THRESHOLD_CRITICAL (за 4 недели)
    - WARNING: payers_1w < THRESHOLD_WARNING (за текущую неделю)
    
    Приоритет: CRITICAL > WARNING
    
    Returns: 'CRITICAL', 'WARNING', or None
    """
    if payers_4w < THRESHOLD_CRITICAL:
        return 'CRITICAL'
    elif payers_1w < THRESHOLD_WARNING:
        return 'WARNING'
    else:
        return None


# --- 5. Основная функция проверки ---

def run_payers_check():
    
    period_start, period_end, extended_start = get_check_period()
    
    print(f"\n--- Checking Payers7 ---")
    print(f"Текущая неделя (WARNING): {period_start} - {period_end}")
    print(f"4 недели (CRITICAL): {extended_start} - {period_end}")
    
    # SQL Запрос: считаем payers за текущую неделю и за 4 недели
    # Фильтруем только активные сегменты оперирования
    sql_query = f"""
    WITH active_segments AS (
        SELECT DISTINCT operation_segment
        FROM operation_segments.segments_parameters
        WHERE coverage_type = 'Рабочее'
          AND actual
          AND valid_to > CURRENT_DATE
          AND partner_id IN {CONFIG_PARTNERS}
    )
    SELECT
        app_short,
        operation_segment_nm,
        partner_id,
        COUNT(DISTINCT CASE
            WHEN payers_7_cnt = 1 AND install_dt BETWEEN '{period_start}' AND '{period_end}'
            THEN plr_id
        END) as payers_count,
        COUNT(DISTINCT CASE
            WHEN payers_7_cnt = 1
            THEN plr_id
        END) as payers_count_4w,
        SUM(CASE WHEN install_dt BETWEEN '{period_start}' AND '{period_end}' THEN installs_cnt ELSE 0 END) as total_installs,
        SUM(installs_cnt) as total_installs_4w
    FROM core.base_metrics
    WHERE
        install_dt BETWEEN '{extended_start}' AND '{period_end}'
        AND partner_id IN {CONFIG_PARTNERS}
        AND operation_segment_nm IN (SELECT operation_segment FROM active_segments)
    GROUP BY app_short, operation_segment_nm, partner_id
    """
    
    df = env.execute_sql(sql_query)
    
    if df.empty:
        print("  >> Нет данных за указанный период.")
        return df
    
    print(f"  >> Получено строк: {len(df)}")
    
    # Заполняем NULL значения в строковых колонках
    df['app_short'] = df['app_short'].fillna('Unknown')
    df['operation_segment_nm'] = df['operation_segment_nm'].fillna('Unknown')
    df['partner_id'] = df['partner_id'].fillna('Unknown')
    
    # Приводим типы
    df['payers_count'] = pd.to_numeric(df['payers_count'], errors='coerce').fillna(0).astype(int)
    df['payers_count_4w'] = pd.to_numeric(df['payers_count_4w'], errors='coerce').fillna(0).astype(int)
    df['total_installs'] = pd.to_numeric(df['total_installs'], errors='coerce').fillna(0).astype(int)
    df['total_installs_4w'] = pd.to_numeric(df['total_installs_4w'], errors='coerce').fillna(0).astype(int)
    
    # Добавляем partner_name через маппинг
    df['partner_name'] = df['partner_id'].astype(str).map(PARTNER_NAMES).fillna(df['partner_id'])
    
    # Добавляем метаданные
    df['period_start'] = period_start
    df['period_end'] = period_end
    df['extended_start'] = extended_start
    df['threshold_warning'] = THRESHOLD_WARNING
    df['threshold_critical'] = THRESHOLD_CRITICAL
    
    # Определяем тип алерта для каждой строки
    df['alert_type'] = df.apply(lambda row: get_alert_type(row['payers_count'], row['payers_count_4w']), axis=1)
    
    # Флаг наличия алерта (любого типа)
    df['is_alert'] = df['alert_type'].notna()
    
    return df


# --- 6. Запуск проверки ---

df_result = run_payers_check()

if not df_result.empty:
    alerts_df = df_result[df_result['is_alert'] == True].copy()
    
    if not alerts_df.empty:
        alerts_df['check_name'] = ALERT_NAME
        alerts_df['metric_crit_category'] = ALERT_CATEGORY
        alerts_df['date'] = datetime.now()
        
        # Подсчёт по типам
        critical_count = (alerts_df['alert_type'] == 'CRITICAL').sum()
        warning_count = (alerts_df['alert_type'] == 'WARNING').sum()
        
        print(f"\n[{ALERT_CATEGORY.upper()}] Алерты найдены: {len(alerts_df)} (CRITICAL: {critical_count}, WARNING: {warning_count})")
        
        # --- 7. Запись в Redshift ---
        db_cols = [
            'date', 'check_name', 'metric_crit_category',
            'app_short', 'operation_segment_nm', 'partner_id', 'partner_name',
            'payers_count', 'payers_count_4w', 'total_installs', 'total_installs_4w',
            'threshold_warning', 'threshold_critical',
            'alert_type', 'is_alert',
            'period_start', 'period_end', 'extended_start'
        ]
        
        df_to_write = alerts_df[db_cols].copy()
        
        if not df_to_write.empty:
            print(f"Запись {len(df_to_write)} строк в Redshift...")
            env.insert_table_into_rs(df_to_write, RS_TABLE_P7, RS_SCHEMA_P7, 10000)
            print("Успешно записано.")
        
        # --- 8. Slack нотификации ---
        if ALERT_ACTIVE_FLAG != 'Enabled':
            print(f"Нотификация '{ALERT_NAME}' отключена.")
        else:
            # Индикаторы для разных типов алертов
            ICON_CRITICAL = ":red_circle:"           # Красный для CRITICAL
            ICON_WARNING = ":large_yellow_circle:"   # Жёлтый для WARNING

            # Группируем по partner_id (одно сообщение на партнёра)
            unique_partners = alerts_df['partner_id'].unique()

            for t_partner in unique_partners:
                subset = alerts_df[alerts_df['partner_id'] == t_partner]
                
                # Получаем название партнёра
                partner_name = subset['partner_name'].iloc[0]

                # Определяем общий индикатор: CRITICAL если есть хотя бы один CRITICAL
                has_critical = (subset['alert_type'] == 'CRITICAL').any()
                header_icon = ICON_CRITICAL if has_critical else ICON_WARNING

                # Основное сообщение (в канал)
                main_message = f"INCENT.OpEx - {ALERT_NAME} ({ALERT_CATEGORY}): {header_icon} *{partner_name}*"

                # Детали по app и сегментам (в тред)
                thread_lines = []
                # Сортировка: по app_short, затем по alert_type (CRITICAL первым)
                subset_sorted = subset.sort_values(by=['app_short', 'alert_type'], ascending=[True, True])

                for _, row in subset_sorted.iterrows():
                    app = row['app_short']
                    segment = row['operation_segment_nm']
                    alert_type = row['alert_type']

                    if alert_type == 'CRITICAL':
                        icon = ICON_CRITICAL
                        payers = row['payers_count_4w']
                        threshold = THRESHOLD_CRITICAL
                        period_info = "4 нед."
                    else:
                        icon = ICON_WARNING
                        payers = row['payers_count']
                        threshold = THRESHOLD_WARNING
                        period_info = "1 нед."

                    line = f"{icon} [{alert_type}] {app.upper()} / {segment}: {payers} payers ({period_info}, threshold: {threshold})"
                    thread_lines.append(line)

                try:
                    slack = env.SlackNotifier("incent_notifications")

                    # 1. Отправляем основное сообщение (в канал)
                    thread_ts = slack.send_message(main_message)

                    # 2. Отправляем детали по app и сегментам (в тред)
                    if thread_lines:
                        thread_message = "\n".join(thread_lines)
                        slack.send_message(thread_message, thread_ts=thread_ts)

                except Exception as e:
                    print(f"Error sending to Slack: {e}")
        
        # --- 9. Вывод отчёта в Jupyter ---
        display_cols = [
            'date', 'check_name', 'metric_crit_category',
            'app_short', 'operation_segment_nm', 'partner_id', 'partner_name',
            'payers_count', 'payers_count_4w', 'total_installs', 'total_installs_4w',
            'threshold_warning', 'threshold_critical',
            'alert_type',
            'period_start', 'period_end', 'extended_start'
        ]
        
        # Сортировка: CRITICAL сверху
        alerts_display = alerts_df[display_cols].sort_values(by='alert_type', ascending=True)
        styled_df = alerts_display.style.hide(axis='index')
        display(styled_df)
        
    else:
        print(f"Алертов не найдено. Все сегменты выше порогов (WARNING: {THRESHOLD_WARNING} за 1 нед., CRITICAL: {THRESHOLD_CRITICAL} за 4 нед.).")
else:
    print("Нет данных для анализа.")